<a href="https://colab.research.google.com/github/adham21ayman/Brain_Tumor/blob/main/brainTumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
TRAIN_DIR = "/content/drive/MyDrive/Colab Notebooks/Brain Tumor/Data/Training"
TEST_DIR = "/content/drive/MyDrive/Colab Notebooks/Brain Tumor/Data/Testing"
CLASS_NAME = ['no_tumor', 'benign_tumor', 'malignant_tumor']

In [ ]:
train , val = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    labels='inferred',
    label_mode='int',
    class_names=CLASS_NAME,
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=21,
    validation_split=0.2,
    subset="both",
)

Found 2870 files belonging to 3 classes.
Using 2296 files for training.
Using 574 files for validation.


In [ ]:
test = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    labels='inferred',
    label_mode='int',
    class_names=CLASS_NAME,
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=21,
)

Found 320 files belonging to 3 classes.


In [ ]:
auto = tf.data.AUTOTUNE
train = train.prefetch(auto)
val = val.prefetch(auto)
test = test.prefetch(auto)

In [ ]:
base_model = tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224,3),
    )

219055592/219055592 [==============================] - 1s 0us/step


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Define input shape
input_shape = (224, 224, 3)
inputs = tf.keras.Input(shape=input_shape)

# Preprocessing layers
x = tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255)(inputs)
x = tf.keras.layers.experimental.preprocessing.Resizing(224, 224)(x)

# Data augmentation layers
x = tf.keras.layers.experimental.preprocessing.RandomFlip(mode='horizontal')(x)
x = tf.keras.layers.experimental.preprocessing.RandomRotation(factor=0.2)(x)
x = tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=0.2, width_factor=0.2)(x)

# Base model
x = base_model(x)

# Flatten and dense layers
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(rate=0.3)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(rate=0.25)(x)

# Output layer
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

# Define the model
model = tf.keras.Model(inputs, outputs)


In [ ]:
for layer in base_model.layers:
    layer.trainable = True

In [ ]:
#after 10 epoch the learn rate will be (e**-1)*LR after evry epoch
def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

In [ ]:
learn_rate_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=4,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=10
)

In [ ]:
model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history_model= model.fit(train, epochs=40,validation_data=val,callbacks=[learn_rate_callback,stopping_callback])

Epoch 1/40
72/72 [==============================] - 124s 624ms/step - loss: 0.1026 - accuracy: 0.9769 - val_loss: 0.4589 - val_accuracy: 0.8885 - lr: 0.0010
Epoch 2/40
72/72 [==============================] - 45s 614ms/step - loss: 0.0927 - accuracy: 0.9691 - val_loss: 2.6306 - val_accuracy: 0.8815 - lr: 0.0010
Epoch 3/40
72/72 [==============================] - 43s 585ms/step - loss: 0.0904 - accuracy: 0.9726 - val_loss: 0.1779 - val_accuracy: 0.9425 - lr: 0.0010
Epoch 4/40
72/72 [==============================] - 43s 581ms/step - loss: 0.0496 - accuracy: 0.9830 - val_loss: 0.1092 - val_accuracy: 0.9617 - lr: 0.0010
Epoch 5/40
72/72 [==============================] - 43s 583ms/step - loss: 0.0561 - accuracy: 0.9830 - val_loss: 0.4065 - val_accuracy: 0.9164 - lr: 0.0010
Epoch 6/40
72/72 [==============================] - 43s 584ms/step - loss: 0.0434 - accuracy: 0.9852 - val_loss: 0.4423 - val_accuracy: 0.9007 - lr: 0.0010
Epoch 7/40
72/72 [==============================] - 43s 580ms/s

In [ ]:
model.evaluate(test)

10/10 [==============================] - 2s 164ms/step - loss: 3.3538 - accuracy: 0.7469


[3.3537871837615967, 0.746874988079071]

In [ ]:
# model.save('/content/drive/MyDrive/Colab Notebooks/Brain Tumor/model/brain768.h5')